# Investigation: AWS S3 Public Access Block Removed

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type="aws:cloudtrail"
eventSource:(
  "s3.amazonaws.com"
  "s3control.amazonaws.com"
)
eventName:(
  DeletePublicAccessBlock
  PutPublicAccessBlock
  DeleteBucketPublicAccessBlock
  PutBucketPublicAccessBlock
  DeleteAccountPublicAccessBlock
  PutAccountPublicAccessBlock
)
not errorCode:*
(
  eventName:(
    DeletePublicAccessBlock
    DeleteBucketPublicAccessBlock
    DeleteAccountPublicAccessBlock
  )
  or
  (
    eventName:(
      PutPublicAccessBlock
      PutBucketPublicAccessBlock
      PutAccountPublicAccessBlock
    )
    (
      requestParameters.PublicAccessBlockConfiguration.BlockPublicAcls:"false"
      or requestParameters.PublicAccessBlockConfiguration.IgnorePublicAcls:"false"
      or requestParameters.PublicAccessBlockConfiguration.BlockPublicPolicy:"false"
      or requestParameters.PublicAccessBlockConfiguration.RestrictPublicBuckets:"false"
    )
  )
)
| groupbycount(
    recipientAccountId,
    userIdentity.arn,
    awsRegion
  )
| where @q.count > 0
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()